# 네트워크 분석 따라해보기;;;

# 대분류, 중분류를 구매 패턴을 통해서 무언가 발견할 수 있지 않을까 에 대한 의문

- 유저아이디, 오더아이디 기준, add_to_cart_order 순서에 따른 엣지 생성 -> 노드화

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [3]:
orders_df = pd.read_csv('orders.csv', dtype = 
                        {'order_id' : np.int32, 'user_id' : np.int32,
                        'eval_set' : 'category',
                        'order_number' : np.int16,
                        'order_dow' : np.int8,
                        'order_hour_of_day' : np.int8,
                        'days_since_prior_order' : np.float32})

products = pd.read_csv('products.csv', dtype={
        'product_id': np.uint16,
        'order_id': np.int32,
        'aisle_id': np.uint8,
        'department_id': np.uint8})

print('loading prior')
priors = pd.read_csv('order_products__prior.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print('loading train')
train = pd.read_csv('order_products__train.csv', dtype={
            'order_id': np.int32,
            'product_id': np.uint16,
            'add_to_cart_order': np.int16,
            'reordered': np.int8})

print ('loading')
aisles = pd.read_csv('aisles.csv')
departments = pd.read_csv('departments.csv')

loading prior
loading train
loading


In [4]:
orders = orders_df[orders_df['eval_set']=='prior']

In [5]:
orders = orders.drop('eval_set', axis = 1)

In [6]:
orders

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0
5,3367565,1,6,2,7,19.0
6,550135,1,7,1,9,20.0
7,3108588,1,8,1,14,14.0
8,2295261,1,9,1,16,0.0
9,2550362,1,10,4,8,30.0


In [8]:
df_one = pd.merge(priors, products, on = 'product_id')

In [9]:
df_two = pd.merge(df_one, orders, on = 'order_id')

In [10]:
df_two.to_csv('merge_order_prior')

In [11]:
df_two.head(5)

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,Organic Egg Whites,86,16,202279,3,5,9,8.0
1,2,28985,2,1,Michigan Organic Kale,83,4,202279,3,5,9,8.0
2,2,9327,3,0,Garlic Powder,104,13,202279,3,5,9,8.0
3,2,45918,4,1,Coconut Butter,19,13,202279,3,5,9,8.0
4,2,30035,5,0,Natural Sweetener,17,13,202279,3,5,9,8.0


# 여기까지는 전체적인 걸 보기 위한 로딩, 머지 작업
# 아래 부터 네트워크 설정 

In [12]:
df_three = df_two[['order_id', 'add_to_cart_order', 'aisle_id', 'department_id', 'user_id']]

In [17]:
departments.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [20]:
df_four = pd.merge(df_three, aisles, on = 'aisle_id')

In [22]:
df_five = pd.merge(df_four, departments, on = 'department_id')

In [26]:
df_five.sort_values(['order_id', 'add_to_cart_order'], inplace = True)

In [80]:
df_five[df_five['order_id'] == 2]

,order_id,add_to_cart_order,aisle_id,department_id,user_id,aisle,department
0,2,1,86,16,202279,eggs,dairy eggs
5414016,2,2,83,4,202279,fresh vegetables,produce
14893307,2,3,104,13,202279,spices seasonings,pantry
15105399,2,4,19,13,202279,oils vinegars,pantry
15350865,2,5,17,13,202279,baking ingredients,pantry
5414017,2,6,83,4,202279,fresh vegetables,produce
15677557,2,7,105,13,202279,doughs gelatins bake mixes,pantry
15783646,2,8,88,13,202279,spreads,pantry
8832037,2,9,123,4,202279,packaged vegetables fruits,produce


In [74]:
exe = {}

In [36]:
df_five_exe = df_five[:1000]

In [71]:
df_six = df_five_exe.groupby('order_id')['aisle']

In [67]:
for name, group in df_five_exe['aisle'].groupby(df_five_exe['order_id']):
    print (name)
    print ('******')
    print (group.values)

2
******
['eggs' 'fresh vegetables' 'spices seasonings' 'oils vinegars'
 'baking ingredients' 'fresh vegetables' 'doughs gelatins bake mixes'
 'spreads' 'packaged vegetables fruits']
3
******
['yogurt' 'soy lactosefree' 'packaged vegetables fruits'
 'packaged vegetables fruits' 'soy lactosefree' 'fresh vegetables'
 'poultry counter' 'bread']
4
******
['breakfast bakery' 'cold flu allergy' 'energy granola bars'
 'breakfast bars pastries' 'breakfast bars pastries'
 'breakfast bars pastries' 'breakfast bars pastries' 'chips pretzels'
 'trail mix snack mix' 'crackers' 'refrigerated' 'energy sports drinks'
 'energy sports drinks']
5
******
['fresh fruits' 'salad dressing toppings' 'prepared soups salads'
 'packaged vegetables fruits' 'milk' 'paper goods'
 'water seltzer sparkling water' 'kosher foods' 'packaged poultry'
 'packaged cheese' 'instant foods' 'packaged produce' 'cookies cakes'
 'candy chocolate' 'fresh vegetables' 'spreads' 'paper goods'
 'cookies cakes' 'body lotions soap' 'pac

# 그룹바이 포문 결과값 해석
- name = order_id
- list = 해당 유저가 산 물건 리스트(오더 카드 순서대로)

In [75]:
for name, group in df_six:
    exe[name] =  group.values

In [76]:
print (exe)

{2: array(['eggs', 'fresh vegetables', 'spices seasonings', 'oils vinegars',
       'baking ingredients', 'fresh vegetables',
       'doughs gelatins bake mixes', 'spreads',
       'packaged vegetables fruits'], dtype=object), 3: array(['yogurt', 'soy lactosefree', 'packaged vegetables fruits',
       'packaged vegetables fruits', 'soy lactosefree', 'fresh vegetables',
       'poultry counter', 'bread'], dtype=object), 4: array(['breakfast bakery', 'cold flu allergy', 'energy granola bars',
       'breakfast bars pastries', 'breakfast bars pastries',
       'breakfast bars pastries', 'breakfast bars pastries',
       'chips pretzels', 'trail mix snack mix', 'crackers', 'refrigerated',
       'energy sports drinks', 'energy sports drinks'], dtype=object), 5: array(['fresh fruits', 'salad dressing toppings', 'prepared soups salads',
       'packaged vegetables fruits', 'milk', 'paper goods',
       'water seltzer sparkling water', 'kosher foods', 'packaged poultry',
       'packaged chee

# 위 두개 그룹바이 포문이 같은 역할을 함.
# 확인 하기 위해서 돌려봤습니다..

In [51]:
import networkx as nx
G = nx.MultiGraph()

In [77]:
def pratice(argu):
    pra = []
    for i in (argu):
        num = len(argu[i])
        for n in range(num):
            if n != num - 1: 
                pra.append([argu[i][n], argu[i][n+1]])
            else:
                pass
    return pra

# 위 함수의 역할은 다음과 같습니다
- 현재 위 exe의 딕트 문의 구성은 이렇습니다(키 : 밸류)
    - 키 = order_id, 밸류 = order_id에 따른 물건 리스트(오더 순서대로)
- 따라서 각 키별 밸류값들을 뽑고, 그 밸류를 두개씩 짝짓는 함수 입니다
    - 예를 들어, order_id가 2 이고, 물건이 사탕, 노트북, 과자 라고 하면
        (사탕, 노트북), (노트북, 과자) 이렇게 묶이도록 해놨습니다.

In [78]:
pratice(exe)

[['eggs', 'fresh vegetables'],
 ['fresh vegetables', 'spices seasonings'],
 ['spices seasonings', 'oils vinegars'],
 ['oils vinegars', 'baking ingredients'],
 ['baking ingredients', 'fresh vegetables'],
 ['fresh vegetables', 'doughs gelatins bake mixes'],
 ['doughs gelatins bake mixes', 'spreads'],
 ['spreads', 'packaged vegetables fruits'],
 ['yogurt', 'soy lactosefree'],
 ['soy lactosefree', 'packaged vegetables fruits'],
 ['packaged vegetables fruits', 'packaged vegetables fruits'],
 ['packaged vegetables fruits', 'soy lactosefree'],
 ['soy lactosefree', 'fresh vegetables'],
 ['fresh vegetables', 'poultry counter'],
 ['poultry counter', 'bread'],
 ['breakfast bakery', 'cold flu allergy'],
 ['cold flu allergy', 'energy granola bars'],
 ['energy granola bars', 'breakfast bars pastries'],
 ['breakfast bars pastries', 'breakfast bars pastries'],
 ['breakfast bars pastries', 'breakfast bars pastries'],
 ['breakfast bars pastries', 'breakfast bars pastries'],
 ['breakfast bars pastries', 

# 노드를 주기 위해 aisle 의 밸류를 그대로 넣기 위한 리스트 입니다

In [57]:
list_aisle = list(aisles['aisle'].values)

In [58]:
list_aisle

['prepared soups salads',
 'specialty cheeses',
 'energy granola bars',
 'instant foods',
 'marinades meat preparation',
 'other',
 'packaged meat',
 'bakery desserts',
 'pasta sauce',
 'kitchen supplies',
 'cold flu allergy',
 'fresh pasta',
 'prepared meals',
 'tofu meat alternatives',
 'packaged seafood',
 'fresh herbs',
 'baking ingredients',
 'bulk dried fruits vegetables',
 'oils vinegars',
 'oral hygiene',
 'packaged cheese',
 'hair care',
 'popcorn jerky',
 'fresh fruits',
 'soap',
 'coffee',
 'beers coolers',
 'red wines',
 'honeys syrups nectars',
 'latino foods',
 'refrigerated',
 'packaged produce',
 'kosher foods',
 'frozen meat seafood',
 'poultry counter',
 'butter',
 'ice cream ice',
 'frozen meals',
 'seafood counter',
 'dog food care',
 'cat food care',
 'frozen vegan vegetarian',
 'buns rolls',
 'eye ear care',
 'candy chocolate',
 'mint gum',
 'vitamins supplements',
 'breakfast bars pastries',
 'packaged poultry',
 'fruit vegetable snacks',
 'preserved dips spreads

In [81]:
G.add_nodes_from(list_aisle)

In [82]:
G.add_edges_from(pratice(exe))

In [83]:
G.edges()

[('prepared soups salads', 'soup broth bouillon'),
 ('prepared soups salads', 'soup broth bouillon'),
 ('prepared soups salads', 'soup broth bouillon'),
 ('prepared soups salads', 'soup broth bouillon'),
 ('prepared soups salads', 'salad dressing toppings'),
 ('prepared soups salads', 'salad dressing toppings'),
 ('prepared soups salads', 'packaged vegetables fruits'),
 ('prepared soups salads', 'packaged vegetables fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'fresh fruits'),
 ('energy granola bars', 'spices seasonings'),
 ('energy granola bars', 'spices seasonings'),
 ('energy granola bars', 'crackers'),
 ('energy granola bars', 'crackers'),
 ('energy granola bars', 'fresh vegetables'),
 ('energy granola bars', 'fresh vegetables'),
 ('energy granola bars', 'spreads'),
 ('energy granola b

In [84]:
with open("exe2_first.graphml", "wb") as graph:
    nx.write_graphml(G, graph)
# nx.write 등으로 저장한다 그래프를

In [85]:
ls *.graphml

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 1086-4A98

 C:\Users\jakethelazy\Desktop\github\pro_instacart 디렉터리

2017-07-20  오후 09:40            96,614 exe_first.graphml
2017-07-20  오후 10:19           188,347 exe2_first.graphml
               2개 파일             284,961 바이트
               0개 디렉터리  45,837,295,616 바이트 남음
